# Capstone Project - The Battle of the Neighborhoods (Week 2)

# Applied Data Science Capstone by IBM/Coursera

# Coursera Capstone - REPORT CONTENT
1. Introduction Section : ⁃ Discussion of the business problem and the interested audience in this project.
2. Data Section:
 ⁃ Description of the data that will be used to solve the problem and the sources.
3. Methodology section ⁃ Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if 
   any machine learnings were used establishing the strategy and purposes.   
4. Results section ⁃ Discussion of the results.
5. Discussion section ⁃ Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6. Conclusion section ⁃ Report Conclusion.

# Description of the Problem and Background

### Scenario:

I am a data scientist residing in Rapid City,South Dakota. 
I currently work with a popular travel agency in the city as the resident data scientist where I make recommendations on proposed tourist sites for our various clients with travel destination to almost anywhere in the world. 
A couple of Italians living in my city are going for a conference in London,United Kingdom for a few days and upon returning to the United States will like to visit the windy city, Chicago,Illinois for about a week.
With hotels already booked in the two cities, I have been saddled with the responsibilities of making the trip to London and Chicago memorable for the intending tourist.The requests of the tourist include finding locations around the hotel that has the following places of interest in London:
 1.  High-end Boutiques
 2.  Italian Resturants
 3.  Intercontinental Resturants
 4.  Departmental stores
 
For  the Chicago location, clients wants the 3 best Italian resturants around the proposed location.
 
In order to make a good recommendations, I have decided to use the skills I leanrt in my IBM Data Science program to solve the requests.

# Business Problem:
The challenge is to find good tourist sites for our clients based on their proposals for London and Chicago.



# Interested Audience
I believe this is a relevant challenge with valid questions and answers for my clients and anyone intending to go on holiday in London and Chicago. Going on
hoidays without adequate informations can pose unforseen problems and also make the visit(s) not enjoyable.

# 2. Data Section:
Description of the data and its sources that will be used to solve the problem

# Description of the Data:

The following data is required to  answer the issues of the problem and allow for proper decision making:

- List of Italian Resturants and the distances with their geodata (latitude and longitude) 
- List of Intercontinental Resturants with their address location 
- List of high-end boutiques with their addresses 
- Rating of the three closest Italian resturants in the city of Chicago and their addresses
- Map of each neioghborhoods to aid navigation



### How the data will be used to solve the problem

The data will be used as follows:
- Use Foursquare and geopy data to map top 10 venues for all London City neighborhoods and top 30 for City of Chicago neighborhood as they intend to spend more time in Chicago ( as per Course LAB)
- Use foursquare and geopy data to map the location of Italian resturants , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each resturants.
- create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
- Use the foursquare venue search,explore and rating functions to get necessary informations as required and convert to dataframe for use of the cients


The procesing of these DATA will allow to answer the key questions to make a decision:
- what are the closest tourist sites as per the clients request that are close to the clients hotel?
- what are the list of the best Italian resturants in the city of Chicago with further review for best?
- What are the best places for intercontinental dishes closest to the hotel?



# Data Exploration

# Import necessary libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.19.0-py_0       conda-forge

The following packages will be UPDATED:

   

Define Foursquare Credentials and Version

In [4]:
CLIENT_ID = 'LW2TUJ3JH5K054YOOFWFTGE0GEFGDR1ER2HLWYYPHPMAWVCN' # your Foursquare ID
CLIENT_SECRET = 'G42FPYM1BA1RAB4P3PSICOYDVTFLPPGXSAXI4ACMH4I3WJYV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LW2TUJ3JH5K054YOOFWFTGE0GEFGDR1ER2HLWYYPHPMAWVCN
CLIENT_SECRET:G42FPYM1BA1RAB4P3PSICOYDVTFLPPGXSAXI4ACMH4I3WJYV


# The client is staying at The Hotel Chicago and converting address to its latitude and longitude coordinates.

In [5]:
address = '333 N Dearborn St, Chicago, IL 60654'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.8885617 -87.6288229220047


Category search

In [6]:
search_query = 'Italian'
radius = 1000
print(search_query + ' .... OK!')

Italian .... OK!


In [7]:
#Defining the corresponding url
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LW2TUJ3JH5K054YOOFWFTGE0GEFGDR1ER2HLWYYPHPMAWVCN&client_secret=G42FPYM1BA1RAB4P3PSICOYDVTFLPPGXSAXI4ACMH4I3WJYV&ll=41.8885617,-87.6288229220047&v=20180604&query=Italian&radius=1000&limit=30'

Send the GET Request and examine the results

In [8]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5cd249c11ed2196c424706d5'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/steakhouse_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1cc941735',
      'name': 'Steakhouse',
      'pluralName': 'Steakhouses',
      'primary': True,
      'shortName': 'Steakhouse'}],
    'hasPerk': False,
    'id': '41fd7500f964a520501f1fe3',
    'location': {'address': '33 W Kinzie St',
     'cc': 'US',
     'city': 'Chicago',
     'country': 'United States',
     'crossStreet': 'at Dearborn St',
     'distance': 77,
     'formattedAddress': ['33 W Kinzie St (at Dearborn St)',
      'Chicago, IL 60654',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.889071055749454,
       'lng': -87.6294590845703}],
     'lat': 41.889071055749454,
     'lng': -87.6294590845703,
     'postalCode': '60654',
     'state': 'IL'},
    'name': "Harry Caray's Italian Steakhouse",
   

Get relevant part of JSON and transform it into a pandas dataframe

In [9]:
# assign relevant part of JSON to venues
venues = result['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,41fd7500f964a520501f1fe3,33 W Kinzie St,US,Chicago,United States,at Dearborn St,77,"[33 W Kinzie St (at Dearborn St), Chicago, IL ...","[{'lat': 41.889071055749454, 'lng': -87.629459...",41.889071,-87.629459,60654,IL,Harry Caray's Italian Steakhouse,v-1557285313,NaN
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",262899,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/italian-vil...,False,4aa986ccf964a520115420e3,71 W Monroe St,US,Chicago,United States,btwn Dearborn St & Clark St,858,"[71 W Monroe St (btwn Dearborn St & Clark St),...","[{'lat': 41.88092612690439, 'lng': -87.6302496...",41.880926,-87.630250,60603,IL,Italian Village,v-1557285313,133758699
2,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",188,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/als-italian...,False,4a47c881f964a52014aa1fe3,169 W Ontario St,US,Chicago,United States,at Wells,657,"[169 W Ontario St (at Wells), Chicago, IL 6065...","[{'lat': 41.89311401105442, 'lng': -87.6338738...",41.893114,-87.633874,60654,IL,Al's #1 Italian Beef,v-1557285313,NaN
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",304049,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/rpm-italian...,False,4f1322f3e4b0725fed919ce7,52 W Illinois St,US,Chicago,United States,at N Dearborn St,262,"[52 W Illinois St (at N Dearborn St), Chicago,...","[{'lat': 41.89081532118322, 'lng': -87.6297554...",41.890815,-87.629755,60654,IL,RPM Italian,v-1557285313,91125334
4,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",263283,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/bongiornos-...,False,4ec5635e0aaf9233edd25117,405 N Wabash Ave,US,Chicago,United States,NaN,232,"[405 N Wabash Ave, Chicago, IL 60611, United S...","[{'lat': 41.88934252168581, 'lng': -87.6262163...",41.889343,-87.626216,60611,IL,Bongiorno's Italian Deli & Pizzeria,v-1557285313,89371959


Defining information of interest and filtering dataframe

In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry Caray's Italian Steakhouse,Steakhouse,33 W Kinzie St,US,Chicago,United States,at Dearborn St,77,"[33 W Kinzie St (at Dearborn St), Chicago, IL ...","[{'lat': 41.889071055749454, 'lng': -87.629459...",41.889071,-87.629459,60654,IL,41fd7500f964a520501f1fe3
1,Italian Village,Italian Restaurant,71 W Monroe St,US,Chicago,United States,btwn Dearborn St & Clark St,858,"[71 W Monroe St (btwn Dearborn St & Clark St),...","[{'lat': 41.88092612690439, 'lng': -87.6302496...",41.880926,-87.630250,60603,IL,4aa986ccf964a520115420e3
2,Al's #1 Italian Beef,Sandwich Place,169 W Ontario St,US,Chicago,United States,at Wells,657,"[169 W Ontario St (at Wells), Chicago, IL 6065...","[{'lat': 41.89311401105442, 'lng': -87.6338738...",41.893114,-87.633874,60654,IL,4a47c881f964a52014aa1fe3
3,RPM Italian,Italian Restaurant,52 W Illinois St,US,Chicago,United States,at N Dearborn St,262,"[52 W Illinois St (at N Dearborn St), Chicago,...","[{'lat': 41.89081532118322, 'lng': -87.6297554...",41.890815,-87.629755,60654,IL,4f1322f3e4b0725fed919ce7
4,Bongiorno's Italian Deli & Pizzeria,Pizza Place,405 N Wabash Ave,US,Chicago,United States,NaN,232,"[405 N Wabash Ave, Chicago, IL 60611, United S...","[{'lat': 41.88934252168581, 'lng': -87.6262163...",41.889343,-87.626216,60611,IL,4ec5635e0aaf9233edd25117
5,Italian Consulate Chicago,Embassy / Consulate,500 N Michigan Ave,US,Chicago,United States,at E Grand Ave,445,"[500 N Michigan Ave (at E Grand Ave), Chicago,...","[{'lat': 41.891201768799604, 'lng': -87.624776...",41.891202,-87.624776,60611,IL,4c87b29856e037042181b4a3
6,Italian Cultural Institute,Government Building,NaN,US,Chicago,United States,NaN,447,"[Chicago, IL 60611, United States]","[{'lat': 41.89120814718745, 'lng': -87.6247598...",41.891208,-87.624760,60611,IL,4f99dcd6e4b0dee1e8cd6dea
7,Villa Fresh Italian Kitchen Asian Chao,Asian Restaurant,18 W Jackson Blvd,US,Chicago,United States,NaN,1150,"[18 W Jackson Blvd, Chicago, IL 60604, United ...","[{'lat': 41.878233, 'lng': -87.628493, 'label'...",41.878233,-87.628493,60604,IL,4cf3ede78333224b0341188e
8,Torali - Italian Steak,Italian Restaurant,160 E Pearson St,US,Chicago,United States,Michigan,1222,"[160 E Pearson St (Michigan), Chicago, IL 6061...","[{'lat': 41.89845436210819, 'lng': -87.6224268...",41.898454,-87.622427,60611,IL,597ce4e0dd84420c834bb970
9,Italian American Chamber Of Commerce,Office,500 N Michigan Ave,US,Chicago,United States,NaN,420,"[500 N Michigan Ave, Chicago, IL 60611, United...","[{'lat': 41.890708, 'lng': -87.624648, 'label'...",41.890708,-87.624648,60611,IL,506c4c5ae4b02fc3c44149cc


Visualizing the Italian restaurants that are nearby

In [11]:
dataframe_filtered.name

0              Harry Caray's Italian Steakhouse
1                               Italian Village
2                          Al's #1 Italian Beef
3                                   RPM Italian
4           Bongiorno's Italian Deli & Pizzeria
5                     Italian Consulate Chicago
6                    Italian Cultural Institute
7        Villa Fresh Italian Kitchen Asian Chao
8                        Torali - Italian Steak
9          Italian American Chamber Of Commerce
10                      Italian Cultural Center
11                              Victory Italian
12                       The Italian Coffee Bar
13                Firenze - Italian Street Food
14                      San Marino Italian Deli
15                   Volare Ristorante Italiano
16                              Rosebud on Rush
17                                 Mei's Corner
18                                  ItalianExpo
19                                   L'Appetito
20                                    Al

# To explore the closest Italian restaurants 

In [12]:
venue_id = '41fd7500f964a520501f1fe3' # ID of Harry Caray's Italian Steakhouse
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LW2TUJ3JH5K054YOOFWFTGE0GEFGDR1ER2HLWYYPHPMAWVCN&client_secret=G42FPYM1BA1RAB4P3PSICOYDVTFLPPGXSAXI4ACMH4I3WJYV&ll=41.8885617,-87.6288229220047&v=20180604&query=Italian&radius=1000&limit=30'

Send GET request for result the 3 closest 

In [13]:
venue_id = '41fd7500f964a520501f1fe3' # ID of Harry Caray's Italian Steakhouse
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.1


In [14]:
venue_id = '4ec5635e0aaf9233edd25117' # ID of Bongiorno's Italian Deli & Pizzeria
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.6


In [15]:
venue_id = '4f1322f3e4b0725fed919ce7' # ID of RPM Italian
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.2


# Based on the ratings RPM Italian has been recommended

# Exploring popular spots around RPM Italian

In [16]:
latitude = 41.890815
longitude = -87.629755

In [17]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=LW2TUJ3JH5K054YOOFWFTGE0GEFGDR1ER2HLWYYPHPMAWVCN&client_secret=G42FPYM1BA1RAB4P3PSICOYDVTFLPPGXSAXI4ACMH4I3WJYV&ll=41.890815,-87.629755&v=20180604&radius=1000&limit=30'

In [18]:
import requests

In [18]:
results = requests.get(url).json()
'There are {} around RPM Italian.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around RPM Italian.'

Get relevant part of JSON

In [19]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4f1322f3e4b0725fed919ce7-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d110941735',
    'name': 'Italian Restaurant',
    'pluralName': 'Italian Restaurants',
    'primary': True,
    'shortName': 'Italian'}],
  'delivery': {'id': '304049',
   'provider': {'icon': {'name': '/delivery_provider_grubhub_20180129.png',
     'prefix': 'https://fastly.4sqi.net/img/general/cap/',
     'sizes': [40, 50]},
    'name': 'grubhub'},
   'url': 'https://www.grubhub.com/restaurant/rpm-italian-52-w-illinois-st-chicago/304049?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=304049'},
  'id': '4f1322f3e4b0725fed919ce7',
  'location': {'address': '52 W Illinois St',
   'cc': 'US',

Process JSON and convert it to a clean dataframe

In [20]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,RPM Italian,Italian Restaurant,52 W Illinois St,US,Chicago,United States,at N Dearborn St,0,"[52 W Illinois St (at N Dearborn St), Chicago,...","[{'lng': -87.62975543356106, 'lat': 41.8908153...",41.890815,-87.629755,NaN,60654,IL,4f1322f3e4b0725fed919ce7
1,Firecakes Donuts,Donut Shop,68 W Hubbard St,US,Chicago,United States,at N Clark St,114,"[68 W Hubbard St (at N Clark St), Chicago, IL ...","[{'lng': -87.63060477613422, 'lat': 41.8900019...",41.890002,-87.630605,NaN,60654,IL,51094b2ce4b04df5935bccba
2,Beatrix,New American Restaurant,519 N Clark St,US,Chicago,United States,at W Grand Ave,118,"[519 N Clark St (at W Grand Ave), Chicago, IL ...","[{'lng': -87.63087360767099, 'lat': 41.8914802...",41.891480,-87.630874,NaN,60654,IL,51b0b828498ecb8d51e396ea
3,Xoco,Mexican Restaurant,449 N Clark St,US,Chicago,United States,at W Illinois St,96,"[449 N Clark St (at W Illinois St), Chicago, I...","[{'lng': -87.63092510796142, 'lat': 41.8908090...",41.890809,-87.630925,NaN,60654,IL,4a9ff513f964a520b83d20e3
4,Sunda,Sushi Restaurant,110 W Illinois St,US,Chicago,United States,btwn N LaSalle Dr & N Clark St,153,[110 W Illinois St (btwn N LaSalle Dr & N Clar...,"[{'lng': -87.63160263384698, 'lat': 41.8908401...",41.890840,-87.631603,NaN,60654,IL,4b32ad2cf964a520a11125e3
5,Roka Akor,Japanese Restaurant,456 N Clark St,US,Chicago,United States,at W Illinois St,128,"[456 N Clark St (at W Illinois St), Chicago, I...","[{'lng': -87.6312797618737, 'lat': 41.89062395...",41.890624,-87.631280,NaN,60654,IL,4dac89634b22f071eae51772
6,Mastro's Steakhouse,Steakhouse,520 N Dearborn St,US,Chicago,United States,at Grand Ave,69,"[520 N Dearborn St (at Grand Ave), Chicago, IL...","[{'lng': -87.62995719909668, 'lat': 41.8914200...",41.891420,-87.629957,NaN,60654,IL,4c7c25322d3ba143df1b96d0
7,Frontera Grill,Mexican Restaurant,445 N Clark St,US,Chicago,United States,btwn Illinois St & Hubbard St,115,[445 N Clark St (btwn Illinois St & Hubbard St...,"[{'lng': -87.63110093742783, 'lat': 41.8905502...",41.890550,-87.631101,NaN,60654,IL,40b28c80f964a520aafb1ee3
8,Tanta Chicago,Peruvian Restaurant,118 W Grand Ave,US,Chicago,United States,NaN,209,"[118 W Grand Ave, Chicago, IL 60654, United St...","[{'lng': -87.63202122555803, 'lat': 41.8916517...",41.891652,-87.632021,NaN,60654,IL,51f6821e498eb0e342bbf904
9,Siena Tavern,Italian Restaurant,51 West Kinzie Street,US,Chicago,United States,at N Dearborn St,182,"[51 West Kinzie Street (at N Dearborn St), Chi...","[{'lng': -87.62971857956552, 'lat': 41.8891733...",41.889173,-87.629719,NaN,60654,IL,50f58653e4b0e1058164dc7e


To visualize these items on the map around our location

In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

# Exploring holiday site in London-Millennium Hotel London Knightsbridge

In [22]:
address = '17 Sloane St, Knightsbridge, London SW1X 9NU, UK'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.4999356 -0.1602214


In [23]:
neighborhood_latitude1=51.4999356
neighborhood_longitude1=-0.1602214

# Exploring venues

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude1, 
    neighborhood_longitude1, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LW2TUJ3JH5K054YOOFWFTGE0GEFGDR1ER2HLWYYPHPMAWVCN&client_secret=G42FPYM1BA1RAB4P3PSICOYDVTFLPPGXSAXI4ACMH4I3WJYV&v=20180604&ll=51.4999356,-0.1602214&radius=500&limit=100'

In [31]:
# answer display is hidden for report simplification 
answer = requests.get(url).json()
#answer

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venuess = answer['response']['groups'][0]['items']
SGnearby_venuess = json_normalize(venuess) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venuess =SGnearby_venuess.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venuess['venue.categories'] = SGnearby_venuess.apply(get_category_type, axis=1)
# clean columns
SGnearby_venuess.columns = [col.split(".")[-1] for col in SGnearby_venuess.columns]

SGnearby_venuess.shape

(100, 4)

In [34]:
# Venues near current London residence place
SGnearby_venuess.head(10)

,name,categories,lat,lng
0,CHANEL Boutique,Boutique,51.498990,-0.159399
1,The Perfume Hall,Perfume Shop,51.499740,-0.162950
2,Harrods,Department Store,51.499572,-0.162698
3,Burger & Lobster,Burger Joint,51.501677,-0.159466
4,Harrods Food Halls,Gourmet Shop,51.499407,-0.162696
5,Harvey Nichols,Department Store,51.501543,-0.159742
6,Amaya Restaurant,Indian Restaurant,51.499264,-0.157540
7,Sumosan Twiga,Japanese Restaurant,51.498979,-0.159473
8,Gucci,Boutique,51.500138,-0.160062
9,Harry's Dolce Vita,Italian Restaurant,51.498566,-0.162131


In [36]:
neighborhood_latitude1=51.4999356
neighborhood_longitude1=-0.1602214
# create map of Chicago place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(SGnearby_venuess['lat'], SGnearby_venuess['lng'], SGnearby_venuess['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

# Result and Conclusion

Our analysis shows that there is a great number of restaurants in both London city and city of Chicago.

And based on the clients requests we are able to provide detailed information to the clients in order to make there trips to both cities eventfull and memorable.

In the city of Chicago, we are able to recommend to the client the most rated Italian resturant close to the hotel with other options and distances from the hotel should the need arises to explore other locations with the detailed map alongside it.

For London city,based on the client's request for detailed informations on intercontinental dishes and high-end boutiques with departmental stores, we are able to give detailed recommendations alongside detailed map for further exploration